In [4]:
from itertools import combinations

In [7]:
import json
import copy

# Load LoRAs configuration
with open('lora_db.json') as file:
    loras_db = json.load(file)

In [11]:
loras = list(loras_db.keys())

In [17]:
import pandas as pd

In [22]:
def combinations_2(loras, n):
    for lora in loras:
        for rest_of_loras in combinations(loras[1:], n -1):
            yield [lora] + list(rest_of_loras)

# pd.DataFrame(list(combinations_2(loras, 3))).to_csv('test.csv')

In [20]:
config_template = {
    "loras": [],
    "weights": [],
    "prompts": [""],
    "trials": 1,
    'images_path': '',
    'seed': 42,
    'pipeline_kwargs': {
        'num_inference_steps': 1,
        "guidance_scale": 0,
    }
}


In [21]:
def generate_configs(loras_db, num_loras):
    configs = []

    for loras_tuple in combinations_2(loras, num_loras):
        main_lora = loras_tuple[0]
        other_loras = loras_tuple[1:]
        
        config = copy.deepcopy(config_template)
        config['loras'] = [main_lora] + other_loras
        config['weights'] = [1] * num_loras

        config['prompts'] = [loras_db[main_lora]['triggers'][0]] if loras_db[main_lora]['triggers'] else [""]

        config['images_path'] = f"./output/{main_lora.lower()}-{'-'.join(other_loras).lower()}/"

        configs.append(config)

    all_configs_path = f'./all_configs_exact_{num_loras}.json'
    with open(all_configs_path, 'w') as file:
        json.dump(configs, file, indent=4)

    print(f"All configurations saved to {all_configs_path}.")


num_loras = int(input("Enter the number of LoRAs to include in each configuration: "))
generate_configs(loras_db, num_loras)

Enter the number of LoRAs to include in each configuration: 3
All configurations saved to ./all_configs_exact_3.json.
